In [11]:
import sys
sys.path.append('..')
from api.api import PDTB
from constituent import Constituent
import itertools

In [12]:
pdtb2 = '/home/pengfei/data/pdtb_v2/all/conll/'
pdtb3 = PDTB(pdtb2)

## only for one individual sentence for explicit

In [14]:
def get_constituents(docid, sentid, conn_index):
    """
    Args:
            docid(str)
            sentid(int)
            conn_index(list[int])
    Returns:
            [ [(token_index, text),...], ...]"""
    syntax_tree = pdtb3.get_syntax_tree(docid, sentid)
    if syntax_tree.tree == None:
        return []
    all_leaves = syntax_tree.tree.get_leaves()
    conn_indices = conn_index
    constituent_nodes = []
    if len(conn_indices) == 1:# like and or so...
        conn_node = syntax_tree.get_leaf_node_by_token_index(conn_indices[0]).up
    else:
        conn_node = syntax_tree.get_common_ancestor_by_token_indices(conn_indices)
        conn_leaves = set([all_leaves.index(syntax_tree.get_leaf_node_by_token_index(conn_index)) for conn_index in conn_indices])
        children = conn_node.get_children()
        for child in children:
            leaves = set([all_leaves.index(n) for n in child.get_leaves()])
            if list(leaves-conn_leaves)!=[]: constituent_nodes.append(list(leaves-conn_leaves))
    curr = conn_node
    while not curr.is_root():
        sibs = [ [all_leaves.index(n) for n in sib] for sib in syntax_tree.get_siblings(curr)]
        constituent_nodes.extend(sibs)
        curr = curr.up

    tokens_indices_with_text = pdtb3.get_tokens_indices_with_text(docid, sentid)
    subtree_list = [ [(o,tokens_indices_with_text[o][2]) for o in node] for node in constituent_nodes]
    return subtree_list

def merge_consti(subtree_list):
    return {(o[0][0],o[-1][0]):i for i,o in enumerate(subtree_list)}

def arg_is_contained(i, arg):
    docid = pdtb3.parse_data[i]['DocID']
    sentid = pdtb3.get_arg_sent_id(i, arg)
    conn_indices = [o[1] for o in pdtb3.get_arg_token_list(i, 'Connective')]
    subtree_list = get_constituents(docid, sentid[0], conn_indices)
    print(len(subtree_list))
    merged_consti_list = merge_consti(subtree_list)
    token_list = pdtb3.get_arg_token_list(i, arg)
    start,end = token_list[0][1], token_list[-1][1]
    # filter results
    consti_list = [k for k in merged_consti_list.keys() if k[0]>=start and k[1]<=end]
    # generate results
    results = []
    for i in range(1, len(consti_list)+1):
        for subset in itertools.combinations(consti_list, i):
            results.append(expand(subset))
    # check
    return [o[1] for o in token_list] in results

def expand(subset):
    ret = []
    for subsubset in subset:
        for i in range(subsubset[0], subsubset[1]+1):
            ret.append(i)
    return sorted(ret)

In [15]:
bad = 0
total =0
for i in range(len(pdtb3.parse_data_test)):
    if pdtb3.parse_data[i]['Type'] == 'Explicit' and pdtb3.get_arg_sent_id(i,'Arg1')[0] == pdtb3.get_arg_sent_id(i,'Arg2')[0]:
        total+=1
        if not arg_is_contained(i, 'Arg1'):
            bad += 1
#             print(pdtb3.get_highlighted_parsetree(i, v=True))
1- bad/total

14
9
6
6
14
4
11
17
7
7
11
4
5
7
4
3
10
8
9
10
14
5
5
6
10
4
9
4
8
3
6
5
8
13
3
8
4
9
11
12
10
4
5
4
5
5
13
5
11
6
11
9
21
6
5
7
6
12
15
9
9
11
7
9
14
6
5
7
4
14
3
6
8
7
8
7
9
6
5
9
10
7
11
3
6
10
11
12
8
4
10
7
6
4
4
5
6
6
3
15
5
11
8
11
6
7
17
11
6
13
18
6
11
5
9
9
4
8
12
8
11
6
4
4
11
8
8
10
16
15
7
11
5
11
5
7
6
3
6
4
5
4
5
5
13
4
4
5
10
12
5
14
5
7
4
6
7
5
10
3
7
4
10
15
18
4
11
7
5
7
5
14
4
6
4
11
5
14
10
6
4
5
5
14
10
7
8
13
0
6
10
12
7
10
5
8
13
9
14
9
8
8
10
11
7
7
5
9
6
5
9
9
16
16
23
10
13
11
10
4
7
16
4
6
5
13
3
7
3
14
9
12
9
5
13
10
5
6
9
4
10
6
18
16
10
15
3
5
10
5
5
10
8
6
4
9
9
14
8
4
6
4
4
0
4
10
12
6
5
10
9
7
16
4
15
7
5
6
7
4
9
8
4
4
4
4
14
7
5
12
10
5
4
5
13
6
7
8
5
10
9
4
5
11
5
6
6
4
14
11
7
4
4
6
4
7
7
0
15
5
5
6
10
9
8
7
12
9
4
11
8
7
13
8
5
8
8
6
5
6
5
4
4
12
7
11
4
10
4
0
0
4
11
4
9
4
4
5
14
6
10
8
24
4
7
5
6
4
7
13
9
7
4
20
15
5
11
5
4
4
7
8
0
9
19
10
5
0
12
4
15
4
5
9
13
8
3
16
8
25
18
11
6
17
7
5
11
12
7
7
9
4
7
7
6
11
5
5
5
13
0
6
6
9
12
8
10
10
5
7
10
13


0.8715415019762845

In [10]:
bad = 0
total =0
for i in range(len(pdtb3.parse_data_test)):
    if pdtb3.parse_data[i]['Type'] == 'Explicit' and pdtb3.get_arg_sent_id(i,'Arg1')[0] == pdtb3.get_arg_sent_id(i,'Arg2')[0]:
        total+=1
        if not arg_is_contained(i, 'Arg2'):
            bad += 1
#             print(pdtb3.get_highlighted_parsetree(i, v=True))
1- bad/total

0.9189723320158103

In [8]:
# only for one individual sentence for implicit
# for PS case for explicit
# for PS case for implicit